Gossiping-QA-Dataset : https://www.kaggle.com/zake7749/pttgossipingcorpus

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('Gossiping-QA-Dataset-2_0.csv')

In [3]:
df.head()

,question,answer
0,為什麼 聖結石 會被酸而 這群人 不會？,質感 劇本 成員 都差很多好嗎 不要拿腎結石來污辱這群人
1,為什麼慶祝228會被罵可是慶端午不會？,因為屈原不是台灣人，是楚國人。
2,有沒有戰神阿瑞斯的八卦?,爵士就是阿瑞斯 男主角最後死了
3,理論與實務最脫節的系,哪個系不脫節...你問最不脫節的簡單多了...
4,為什麼PTT這麼多人看棒球,肥宅才看棒球 系壘一堆胖子


In [4]:
print(len(df))
df = df[df['question'].notnull()]
df = df[df['answer'].notnull()]
df = df[df['question'].notna()]
df = df[df['answer'].notna()]
print(len(df))

774114
774108


In [5]:
df['question'][0]

'為什麼 聖結石 會被酸而 這群人 不會？'

In [6]:
df['answer'][0]

'質感 劇本 成員 都差很多好嗎 不要拿腎結石來污辱這群人'

In [7]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import BertTokenizer, BertModel

import numpy as np
import random
import math

In [8]:
g_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
g_tokenizer = BertTokenizer.from_pretrained("bert-base-chinese") #'bert-base-uncased'
g_vocab_size = g_tokenizer.vocab_size
g_max_input_length = g_tokenizer.max_model_input_sizes['bert-base-chinese']  # 512
g_bert = BertModel.from_pretrained('bert-base-chinese')
g_bert_emb_dim = g_bert.config.to_dict()['hidden_size']

In [10]:
def tokenize_and_cut(sentence):
    tokens = g_tokenizer.tokenize(sentence)
    tokens = tokens[:g_max_input_length-2]
    return tokens

In [11]:
g_vocab_size

21128

In [12]:
class QADataset(Dataset):
    def __init__(self, train_Q, train_A):
        
        self.train_Q = train_Q
        self.train_A = train_A
        self.length = len(train_Q)
        
    def __getitem__(self, index):
           
#         Q_token = tokenize_and_cut("[CLS] " + str(self.train_Q[index]) + " [SEP]")
#         A_token = tokenize_and_cut("[CLS] " + str(self.train_A[index]) + " [SEP]")

        Q_token = tokenize_and_cut(str(self.train_Q[index]))
        A_token = tokenize_and_cut(str(self.train_A[index]+"。"))
        
        Q_index = g_tokenizer.convert_tokens_to_ids(Q_token)
        A_index = g_tokenizer.convert_tokens_to_ids(A_token)
        
        Q_tensor = torch.tensor([Q_index]).transpose(0, 1).to(g_device)
        A_tensor = torch.tensor([A_index]).transpose(0, 1).to(g_device)
        
        return Q_tensor, A_tensor

    def __len__(self):

        return self.length

In [13]:
train_set = QADataset(df['question'], df['answer'])

In [14]:
train_set.__getitem__(0)

(tensor([[4158],
         [ 784],
         [7938],
         [5469],
         [5178],
         [4767],
         [3298],
         [6158],
         [7000],
         [5445],
         [6857],
         [5408],
         [ 782],
         [ 679],
         [3298],
         [8043]], device='cuda:0'),
 tensor([[6549],
         [2697],
         [1206],
         [3315],
         [2768],
         [1519],
         [6963],
         [2345],
         [2523],
         [1914],
         [1962],
         [1621],
         [ 679],
         [6206],
         [2897],
         [5575],
         [5178],
         [4767],
         [ 889],
         [3738],
         [6802],
         [6857],
         [5408],
         [ 782],
         [ 511]], device='cuda:0'))

In [15]:
def create_mini_batch(samples):

    Q_tensor = [s[0] for s in samples]
    A_tensor = [s[1] for s in samples]
    
    Q_tensor = pad_sequence(Q_tensor)
    A_tensor = pad_sequence(A_tensor)
    
#     masks_Q_tensors = torch.zeros(Q_tensor.size(), dtype=torch.long).to(g_device)
#     masks_Q_tensors = masks_Q_tensors.masked_fill(Q_tensor != 0, 1)
    
#     masks_A_tensors = torch.zeros(A_tensor.size(), dtype=torch.long).to(g_device)
#     masks_A_tensors = masks_A_tensors.masked_fill(A_tensor != 0, 1)
    
#     return Q_tensor, A_tensor, masks_Q_tensors, masks_A_tensors
    return Q_tensor, A_tensor

In [16]:
len(df)

774108

In [17]:
temp = 50000
t_set, v_set = torch.utils.data.random_split(train_set, [temp, len(df)-temp])

In [18]:
train_data = DataLoader(t_set, shuffle=True, batch_size=15, collate_fn=create_mini_batch)

model: https://github.com/demi6od/ChatBot/blob/master/image/ChatBotBertTransformer.jpg

In [19]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [20]:
class TransBertEncoder(nn.Module):
    def __init__(self, nhead=8, nlayers=6, dropout=0.5):
        super().__init__()

        # bert encoder
        self.bert = g_bert

        # transformer encoder, as bert last layer fine-tune
        self.pos_encoder = PositionalEncoding(g_bert_emb_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model=g_bert_emb_dim, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)

    def forward(self, src):
        # src = [src len, batch size]

        with torch.no_grad():
            # embedded = [src len, batch size, emb dim]
            embedded = self.bert(src.transpose(0, 1))[0].transpose(0, 1)

        embedded = self.pos_encoder(embedded)
        src_mask = nn.Transformer().generate_square_subsequent_mask(len(embedded)).to(g_device)

        # outputs = [src len, batch size, hid dim * n directions]
        outputs = self.transformer_encoder(embedded, mask = src_mask)

        return outputs

In [21]:
class TransBertDecoder(nn.Module):
    def __init__(self, nhead=8, nlayers=6, dropout=0.5):
        super().__init__()

        self.bert = g_bert

        self.pos_decoder = PositionalEncoding(g_bert_emb_dim, dropout)
        decoder_layer = nn.TransformerDecoderLayer(d_model=g_bert_emb_dim, nhead=nhead)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=nlayers)

        self.fc_out = nn.Linear(g_bert_emb_dim, g_vocab_size)

    def forward(self, tgt, meaning, teacher_forcing_ratio = 0.01):
        # tgt = [output_len, batch size]
        output_len = tgt.size(0)
        batch_size = tgt.size(1)
        
        # teacher_forcing_ratio = 1 -> always true
        teacher_force = random.random() < teacher_forcing_ratio 
        
        if teacher_force and self.training:
            tgt_emb_total = torch.zeros(output_len, batch_size, g_bert_emb_dim).to(g_device)

            for t in range(0, output_len):
                with torch.no_grad():
                    tgt_emb = self.bert(tgt[:t+1].transpose(0, 1))[0].transpose(0, 1)
                        
                tgt_emb_total[t] = tgt_emb[-1]

            tgt_mask = nn.Transformer().generate_square_subsequent_mask(len(tgt_emb_total)).to(g_device)
            decoder_output = self.transformer_decoder(tgt=tgt_emb_total,
                                                      memory=meaning,
                                                      tgt_mask=tgt_mask)
            predictions = self.fc_out(decoder_output)
        else:
            # initialized the input of the decoder with sos_idx (start of sentence token idx)
            output = torch.full((output_len+1, batch_size), g_tokenizer.cls_token_id, dtype=torch.long, device=g_device)
            predictions = torch.zeros(output_len, batch_size, g_vocab_size).to(g_device)

            for t in range(0, output_len):
                with torch.no_grad():
                    # tgt_emb = [t, batch size, emb dim]
                    tgt_emb = self.bert(tgt[:t+1].transpose(0, 1))[0].transpose(0, 1)
                            
                # tgt_emb = self.pos_encoder(tgt_emb)
                tgt_mask = nn.Transformer().generate_square_subsequent_mask(len(tgt_emb)).to(g_device)

                # decoder_output = [t, batch size, emb dim]
                decoder_output = self.transformer_decoder(tgt=tgt_emb,
                                                          memory=meaning,
                                                          tgt_mask=tgt_mask)

                # prediction = [batch size, vocab size]
                prediction = self.fc_out(decoder_output[-1])

                # predictions = [output_len, batch size, vocab size]
                predictions[t] = prediction

                one_hot_idx = prediction.argmax(1)

                # output  = [output len, batch size]
                output[t+1] = one_hot_idx

        return predictions

In [22]:
class GruEncoder(nn.Module):
    """compress the request embeddings to meaning"""

    def __init__(self, hidden_size, input_size):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size)

    def forward(self, input):
        output, hidden = self.gru(input)
        return hidden

In [23]:
class GruDecoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.gru = nn.GRU(output_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, src, tgt, hidden):
        # first input to the decoder is the <CLS> tokens
        fc_output = src[0].unsqueeze(0)
        tgt_len = tgt.size(0)
        batch_size = tgt.size(1)

        # tensor to store decoder outputs
        outputs = torch.zeros(tgt_len, batch_size, g_bert_emb_dim).to(g_device)

        for t in range(0, tgt_len):
            # insert input token embedding, previous hidden state and the context state
            # receive output tensor (predictions) and new hidden state
            gru_output, hidden = self.gru(fc_output, hidden)

            fc_output = self.fc(gru_output)

            # place predictions in a tensor holding predictions for each token
            outputs[t] = fc_output
        return outputs

In [24]:
class DialogDNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.5):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        # ResNet, dropout on first 3 layers
        input = self.dropout(input)

        output = input + F.relu(self.fc1(input))
        output = self.dropout(output)

        output = output + F.relu(self.fc2(output))
        output = self.dropout(output)

        output = output + self.fc3(output)  # no relu to keep negative values

        return output


In [25]:
class Seq2Seq(nn.Module):
    def __init__(self, transbert_encoder, transbert_decoder, gru_encoder, gru_decoder, dialog_dnn):
        super().__init__()

        self.transbert_encoder = transbert_encoder
        self.transbert_decoder = transbert_decoder

        self.gru_encoder = gru_encoder
        self.gru_decoder = gru_decoder

        self.dialog_dnn = dialog_dnn

    def forward(self, src, tgt, teacher_forcing_ratio):
        request_embeddings = self.transbert_encoder(src)
        request_meaning = self.gru_encoder(request_embeddings)

        if TRAIN_DIALOG:
            response_meaning = self.dialog_dnn(request_meaning)
        else:
            response_meaning = request_meaning   

        response_embeddings = self.gru_decoder(request_embeddings, tgt, response_meaning)
        response = self.transbert_decoder(tgt, response_embeddings, teacher_forcing_ratio)

        return response

In [26]:
def print_chat(sentences):
    print("chatbot: ", end="")
    for word_embeds in sentences:
        #pick first
        word_embed = word_embeds[0]
        # find one shot index from word embedding
        max_idx_t = word_embed.argmax()
        max_idx = max_idx_t.item()
        word = g_tokenizer.convert_ids_to_tokens(max_idx)
        print(word, end=" ")
    print("")  # new line at the end of sentence


def print_target_tensor(sentences):
    print("target: ", end="")
    for word_embeds in sentences:
        #pick first
        word_embed = word_embeds[0]
        max_idx = word_embed.item()
        word = g_tokenizer.convert_ids_to_tokens(max_idx)
        print(word, end=" ")
    print("")  # new line at the end of sentence
    
def print_input_tensor(sentences):
    print("input: ", end="")
    for word_embeds in sentences:
        #pick first
        word_embed = word_embeds[0]
        max_idx = word_embed.item()
        word = g_tokenizer.convert_ids_to_tokens(max_idx)
        print(word, end=" ")
    print("")  # new line at the end of sentence

In [27]:
INPUT_DIM = g_vocab_size
OUTPUT_DIM = g_vocab_size
ENC_EMB_DIM = g_bert_emb_dim
DEC_EMB_DIM = g_bert_emb_dim
HID_DIM = 2048  # 5 * 200
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
TRANSFORMER_ENCODER_LAYER = 1  # bert fine-tune
TRANSFORMER_DECODER_LAYER = 3  # semantics -> morphology -> syntax
TRANSFORMER_HEAD = 8

transbert_encoder = TransBertEncoder(TRANSFORMER_HEAD, TRANSFORMER_ENCODER_LAYER, ENC_DROPOUT)
transbert_decoder = TransBertDecoder(TRANSFORMER_HEAD, TRANSFORMER_DECODER_LAYER, DEC_DROPOUT)
gru_encoder = GruEncoder(HID_DIM, ENC_EMB_DIM)
gru_decoder = GruDecoder(HID_DIM, DEC_EMB_DIM)
dialog_dnn = DialogDNN(HID_DIM, HID_DIM, HID_DIM)

In [28]:
g_model = Seq2Seq(transbert_encoder, transbert_decoder, gru_encoder, gru_decoder, dialog_dnn).to(g_device)

In [29]:
g_model

Seq2Seq(
  (transbert_encoder): TransBertEncoder(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(21128, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
    

In [30]:
import torch.optim as optim

In [31]:
optimizer = optim.Adam(g_model.parameters())
loss_fn = nn.CrossEntropyLoss(ignore_index=g_tokenizer.pad_token_id)

In [32]:
len(df)

774108

In [33]:
len(train_data)

3334

In [34]:
iter = len(train_data)

In [35]:
g_model = torch.load('./save/model_test3')

In [37]:
TRAIN_DIALOG = True
for e in range(1): #lucky number
    
    mean_loss = 0
    count = 0
    
    for i, t in train_data:
        
        try:

            optimizer.zero_grad()

            i= i.view(i.shape[0],i.shape[1])
            t = t.view(t.shape[0],t.shape[1])
            
            # src = [src len, batch size]
            # tgt = [output_len, batch size]

            output =  g_model(i, t, 0.5)
            output_dim = output.shape[-1]
            loss = loss_fn(output.view(-1, output_dim), t.view(-1))
            
#             print(output.size())
#             print(output_dim)
#             print(output.view(-1, output_dim).size())
#             print(t.view(-1).size())

            
            loss.backward()
            optimizer.step()

            count += 1
            mean_loss+=loss
     
        except RuntimeError: 
            print('error.....1')
        except AttributeError: 
            print('error.....2')
        else:
            print("e : " + str(e) + ' (' + str(count) + '/' + str(iter) + ') loss : ' + str(float(loss)) )
            if count % 10 == 0:
                print_input_tensor(i)
                print_target_tensor(t)
                print_chat(output)

    print("*****e : "+str(e)+" L : "+str(mean_loss/count)+"*****")
        
    torch.save(g_model, "./model_"+str(e))

e : 0 (1/3334) loss : 7.070329666137695
e : 0 (2/3334) loss : 7.196587085723877
e : 0 (3/3334) loss : 7.10025691986084
e : 0 (4/3334) loss : 7.145488262176514
e : 0 (5/3334) loss : 6.854111671447754
e : 0 (6/3334) loss : 7.39457893371582
e : 0 (7/3334) loss : 6.831321716308594
e : 0 (8/3334) loss : 6.961146354675293
e : 0 (9/3334) loss : 6.7012739181518555
e : 0 (10/3334) loss : 7.174962997436523
input: 肥 宅 穿 雪 靴 會 讓 人 有 小 baby 的 悸 動 嗎 [PAD] [PAD] [PAD] 
target: 像 基 紐 特 戰 隊 吧 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 的 不 不 不 不 不 的 不 的 不 不 就 的 的 不 不 的 的 的 的 不 不 不 
e : 0 (11/3334) loss : 7.284234523773193
e : 0 (12/3334) loss : 6.500909805297852
e : 0 (13/3334) loss : 6.9264960289001465
e : 0 (14/3334) loss : 6.774962425231934
e : 0 (15/3334) loss : 7.118701934814453
e : 0 (16/3334) loss : 6.554360866546631
e : 0 (17/3334) loss : 6.940448760986328
e : 0 (18/3334) loss : 6.590342044830322
e : 0 (19/3334) loss : 6.729740619659424


e : 0 (131/3334) loss : 7.182337760925293
e : 0 (132/3334) loss : 6.659750461578369
e : 0 (133/3334) loss : 7.007812023162842
e : 0 (134/3334) loss : 7.112670421600342
e : 0 (135/3334) loss : 7.104915142059326
e : 0 (136/3334) loss : 6.885367393493652
e : 0 (137/3334) loss : 6.821339130401611
e : 0 (138/3334) loss : 7.105748653411865
e : 0 (139/3334) loss : 7.464900493621826
e : 0 (140/3334) loss : 7.149190902709961
input: 有 沒 有 甲 狀 腺 亢 進 的 八 卦 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 我 之 前 吃 了 兩 年 藥 也 沒 變 胖 啊 反 而 最 近 幾 年 一 直 胖 。 
chatbot: 不 不 不 一 的 的 的 的 的 不 的 不 不 的 不 不 不 不 的 的 的 不 的 
e : 0 (141/3334) loss : 6.978708267211914
e : 0 (142/3334) loss : 6.688735008239746
e : 0 (143/3334) loss : 7.106686115264893
e : 0 (144/3334) loss : 6.850161075592041
e : 0 (145/3334) loss : 6.814092636108398
e : 0 (146/3334) loss : 6.666689395904541
e : 0 (147/3334) loss : 6.942694664001465
e : 0 (148/3334) loss : 6.962291240692139
e : 0 (149/3334) loss : 7.2463483810424805
e : 0 (150/3334) loss : 

e : 0 (261/3334) loss : 7.128744125366211
e : 0 (262/3334) loss : 6.922535419464111
e : 0 (263/3334) loss : 7.1184892654418945
e : 0 (264/3334) loss : 6.980681896209717
e : 0 (265/3334) loss : 6.696938514709473
e : 0 (266/3334) loss : 6.932398319244385
e : 0 (267/3334) loss : 6.863063335418701
e : 0 (268/3334) loss : 6.921705722808838
e : 0 (269/3334) loss : 6.899007320404053
e : 0 (270/3334) loss : 6.875816345214844
input: 館 長 直 播 揍 陳 興 會 怎 樣 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 你 去 問 問 打 [UNK] 的 陳 老 師 還 有 點 機 會 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 不 的 的 的 不 的 的 的 不 不 一 不 的 的 的 的 的 的 不 不 的 的 不 不 的 
e : 0 (271/3334) loss : 6.921394348144531
e : 0 (272/3334) loss : 6.809695243835449
e : 0 (273/3334) loss : 6.7314887046813965
e : 0 (274/3334) loss : 7.2962212562561035
e : 0 (275/3334) loss : 7.207239151000977
e : 0 (276/3334) loss : 7.259853839874268
e : 0 (277/3334) loss : 6.8211588859558105
e : 0 (278/3334) loss : 7.023688316345215
e : 0 (279/3334) 

e : 0 (396/3334) loss : 7.077903747558594
e : 0 (397/3334) loss : 6.6407904624938965
e : 0 (398/3334) loss : 6.791876316070557
e : 0 (399/3334) loss : 6.706681728363037
e : 0 (400/3334) loss : 6.58101224899292
input: 228 連 假 隔 天 大 家 會 去 上 課 嗎 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 幹 當 學 生 真 爽 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 不 的 的 不 的 不 就 的 不 的 的 的 不 的 不 不 的 的 的 不 的 不 不 
e : 0 (401/3334) loss : 7.1890764236450195
e : 0 (402/3334) loss : 6.821101188659668
e : 0 (403/3334) loss : 7.1359710693359375
e : 0 (404/3334) loss : 6.893045425415039
e : 0 (405/3334) loss : 6.903397560119629
e : 0 (406/3334) loss : 7.14539909362793
e : 0 (407/3334) loss : 6.83892297744751
e : 0 (408/3334) loss : 6.763853549957275
e : 0 (409/3334) loss : 6.999514102935791
e : 0 (410/3334) loss : 6.65871000289917
input: 聖 誕 卡 片 接 龍 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 先 有 女 友 再 說 。 [PAD] [PAD] 

e : 0 (521/3334) loss : 6.691389560699463
e : 0 (522/3334) loss : 7.11459493637085
e : 0 (523/3334) loss : 7.058228969573975
e : 0 (524/3334) loss : 7.059263706207275
e : 0 (525/3334) loss : 6.875002384185791
e : 0 (526/3334) loss : 6.971358776092529
e : 0 (527/3334) loss : 6.969923496246338
e : 0 (528/3334) loss : 6.966388702392578
e : 0 (529/3334) loss : 7.007637023925781
e : 0 (530/3334) loss : 6.811379909515381
input: 遊 行 或 是 上 街 後 面 沒 有 籌 備 可 以 辦 得 好 嗎 ? 
target: 應 該 還 是 會 有 總 召 的 收 割 這 種 事 情 不 用 擔 心 沒 人 。 [PAD] 
chatbot: 不 不 不 不 不 的 不 的 的 的 不 不 的 不 不 的 的 不 的 不 不 的 不 
e : 0 (531/3334) loss : 6.718677520751953
e : 0 (532/3334) loss : 6.847116947174072
e : 0 (533/3334) loss : 6.924639701843262
e : 0 (534/3334) loss : 6.93479061126709
e : 0 (535/3334) loss : 6.980344295501709
e : 0 (536/3334) loss : 6.827245712280273
e : 0 (537/3334) loss : 7.222417831420898
e : 0 (538/3334) loss : 6.926293849945068
e : 0 (539/3334) loss : 6.964112281799316
e : 0 (540/3334) loss : 6.6461181640625
inp

...

e : 0 (781/3334) loss : 7.003927230834961
e : 0 (782/3334) loss : 6.8980326652526855
e : 0 (783/3334) loss : 6.914371490478516
e : 0 (784/3334) loss : 7.064876079559326
e : 0 (785/3334) loss : 6.8221821784973145
e : 0 (786/3334) loss : 7.019160747528076
e : 0 (787/3334) loss : 7.331934928894043
e : 0 (788/3334) loss : 7.164636135101318
e : 0 (789/3334) loss : 6.945629596710205
e : 0 (790/3334) loss : 7.055440425872803
input: 鄉 民 要 怎 麼 幫 助 騎 士 贏 球 ？ [PAD] [PAD] [PAD] [PAD] 
target: 我 壓 延 長 賽 . . . 74 快 call 乾 爹 護 駕 阿 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 不 的 不 的 的 不 不 的 不 不 的 的 的 的 的 的 不 的 不 的 不 的 的 的 
e : 0 (791/3334) loss : 7.362661838531494
e : 0 (792/3334) loss : 7.099806785583496
e : 0 (793/3334) loss : 6.959193706512451
e : 0 (794/3334) loss : 6.880521774291992
e : 0 (795/3334) loss : 7.057114124298096
e : 0 (796/3334) loss : 7.0933990478515625
e : 0 (797/3334) loss : 6.772912979125977
e : 0 (798/3334) loss : 7.033262252807617
e : 0 (799/3334) loss : 6.859582424163

e : 0 (912/3334) loss : 6.984925270080566
e : 0 (913/3334) loss : 7.092807292938232
e : 0 (914/3334) loss : 6.923015117645264
e : 0 (915/3334) loss : 7.028698921203613
e : 0 (916/3334) loss : 7.005003452301025
e : 0 (917/3334) loss : 7.031404495239258
e : 0 (918/3334) loss : 6.701136112213135
e : 0 (919/3334) loss : 7.129680633544922
e : 0 (920/3334) loss : 6.883603096008301
input: 吃 屎 哥 是 不 是 賺 到 一 千 萬 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 就 一 堆 人 想 免 費 蹭 熱 度 阿 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 的 不 的 不 的 的 的 的 不 不 的 的 的 不 的 不 不 不 的 不 不 的 
e : 0 (921/3334) loss : 6.997321605682373
e : 0 (922/3334) loss : 6.78893518447876
e : 0 (923/3334) loss : 7.085141658782959
e : 0 (924/3334) loss : 6.652520656585693
e : 0 (925/3334) loss : 7.102802276611328
e : 0 (926/3334) loss : 6.801201820373535
e : 0 (927/3334) loss : 6.679989337921143
e : 0 (928/3334) loss : 6.89788818359375
e : 0 (929/3334) loss : 7.140954971313477
e : 0 (930/3334) loss : 6.8826136

e : 0 (1041/3334) loss : 7.334344863891602
e : 0 (1042/3334) loss : 7.058819770812988
e : 0 (1043/3334) loss : 6.862817287445068
e : 0 (1044/3334) loss : 6.659632205963135
e : 0 (1045/3334) loss : 7.072122573852539
e : 0 (1046/3334) loss : 7.237457752227783
e : 0 (1047/3334) loss : 7.016625881195068
e : 0 (1048/3334) loss : 6.865620136260986
e : 0 (1049/3334) loss : 7.298626899719238
e : 0 (1050/3334) loss : 7.087352275848389
input: 畢 業 典 禮 沒 收 到 花 怎 麼 辦 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
target: 國 小 畢 業 ？ 。 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
chatbot: 不 的 的 不 的 的 的 的 不 的 的 不 的 的 的 的 不 的 
e : 0 (1051/3334) loss : 6.937058448791504
e : 0 (1052/3334) loss : 7.047806262969971
e : 0 (1053/3334) loss : 6.830868244171143
e : 0 (1054/3334) loss : 6.928366184234619
e : 0 (1055/3334) loss : 6.941479206085205
e : 0 (1056/3334) loss : 6.994722843170166
e : 0 (1057/3334) loss : 7.179230213165283
e : 0 (1058/3334) loss : 6.846347808837891
e : 0 (1059/3334) los

e : 0 (1171/3334) loss : 6.828174114227295
e : 0 (1172/3334) loss : 6.598046779632568
e : 0 (1173/3334) loss : 7.10786247253418
e : 0 (1174/3334) loss : 6.7446746826171875
e : 0 (1175/3334) loss : 7.232537269592285
e : 0 (1176/3334) loss : 7.225384712219238


KeyError: 243462

In [52]:
# torch.save(g_model, "./model.pkl")

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Seq2Seq. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type TransBertEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type PositionalEncoding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type TransBertDecoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It wo

In [ ]:
test_data = DataLoader(v_set, shuffle=True, batch_size=1, collate_fn=create_mini_batch)

In [36]:
from torchviz import make_dot, make_dot_from_trace

In [40]:
import os
os.environ["PATH"] += os.pathsep + 'C:\Program Files\Graphviz 2.44.1\bin'
os.environ["PATH"] += os.pathsep + 'C:\ProgramData\Anaconda3\Lib\site-packages\graphviz'
TRAIN_DIALOG = True
x,y = train_set.__getitem__(0)
make_dot(g_model(x,y,1), params=dict(g_model.named_parameters())).render("chatbot", format="png")

'chatbot.png'